In [1]:
import logging
import os
import gzip

import pandas
import easydict

import analysis

In [2]:
analysis.logger.propagate = False

In [3]:
commit = '153688869fb87412dd716ac27994e51baeb16078'
url = 'https://raw.githubusercontent.com/dhimmel/stargeo/{}/data/queries.tsv'.format(commit)
query_df = pandas.read_table(url)
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Nonsmoker == 'Nonsmoker'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC' or PC_tissue_case == 'PC_tissue_case',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = 'data/{}'.format(params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler('{}/log.txt'.format(directory))
    logfile.setLevel(logging.DEBUG)
    analysis.logger.addHandler(logfile)
    sample_df, result_df = analysis.perform_analysis(params)
#     except:
#         sample_df, result_df = None, None
    analysis.logger.removeHandler(logfile)
    if sample_df is not None:
        path = '{}/samples.tsv'.format(directory)
        sample_df.to_csv(path, index=False, sep='\t')
        
    if result_df is not None:
        path = '{}/meta.tsv'.format(directory)
        result_df.to_csv(path, index=False, sep='\t', float_format='%.5g')

/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/rpy/__init__.py:11: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  FutureWarning)


0 DOID:0050156 idiopathic pulmonary fibrosis
1 DOID:0050741 alcohol dependence
2 DOID:0050742 nicotine dependence
3 DOID:1024 leprosy
4 DOID:10283 prostate cancer
5 DOID:10534 stomach cancer
6 DOID:10608 celiac disease
7 DOID:10652 Alzheimer's disease
8 DOID:10763 hypertension


analysis.py:328: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  samples['subset'] = "NA"


ZeroDivisionError: Weights sum to zero, can't be normalized

In [ ]:
#analysis.get_analysis_df(params.case_query, params.control_query, params.modifier_query)